# Problem 3

A company plans its business in a three month period. It can produce 110
units at a cost of 600 each. The minimum amount it must produce per
month is 15 units if active (but of course, it can choose to be closed during
the month, and produce 0 units). Each month it can subcontract the
production of 60 units, at a cost of 660 each. Storing a unit for one month
costs 20$ per unit per month. The marketing department has forecasted
sales of 100, 130 and 150 units for the next three months, respectively.

The goal is to meet the demand each month while minimizing the total
cost.

## Variables

- $u_1$ - number of units with a price of 600 produced in the first month
- $u_2$ - number of units with a price of 600 produced in the second month
- $u_3$ - number of units with a price of 600 produced in the third month

- $su_1$ - number of contracts with 60 units, 660 cost each in first month
- $su_2$ - number of contracts with 60 units, 660 cost each in second month
- $su_3$ - number of contracts with 60 units, 660 cost each in third month





## Objective Function

- Cost for any $u_i, i \in \{1, 2, 3\} = 620$, 600 for production and 20 for storing.
- Cost for any $su_i, i \in \{1, 2, 3\} = 40800$, 60 units per contract, 660 for production of a unit, 20 for storing, so $60 \cdot (660 + 20) = 40800$.

$min((u_1 + u_2 + u_3) \cdot 620 + (su_1 + su_2 + su_3) \cdot 40800)$

## Constraints

- $u_1 + u_2 + u_3 \leq 110$, Maximum Cost of Units with cost 600 Constraint
- $u_1 + 60su_1 \geq 100$, Minimum Units Produced in First Month Constraint
- $u_2 + 60su_2 \geq 130$, Minimum Units Produced in Second Month Constraint
- $u_3 + 60su_3 \geq 150$, Minimum Units Produced in Third Month Constraint

## Code

In [1]:
from pulp import *

max_cheap_units = 110
forecasts = [100, 130, 150]

problem = LpProblem('Problem3', LpMinimize)

# Variables
units_variables = LpVariable.dicts('u', range(1, 4), lowBound=0, cat=LpInteger)
contracts_variables = LpVariable.dicts('su', range(1, 4), lowBound=0, cat=LpInteger)

# Objective Function
problem += lpSum(units_variables) * 620 + lpSum(contracts_variables) * 40800

# Constraints
problem += lpSum(units_variables) <= max_cheap_units, 'Maximum cost of units with cost 600 Constraint'
for i in range(3):
    problem += units_variables[i + 1] + 60 * contracts_variables[i + 1] >= forecasts[i], f'Minimum units produced in month {i + 1} Constraint'

# Solve Problem and Display Results
problem.solve(PULP_CBC_CMD(msg=0))

if LpStatus[problem.status] == "Optimal":
    print(f'Total cost: ${value(problem.objective)}\n')

for variable in problem.variables():
    print(f'{variable.name}: {variable.varValue}')


Total cost: $253600.0

su_1: 1.0
su_2: 2.0
su_3: 2.0
u_1: 40.0
u_2: 10.0
u_3: 30.0


# Problem 4

A consulting company estimates its business for the next five months, in hours, at 6000, 7000, 8000, 9500, 11000 hours, respectively. Currently the company employs 50 consultants, each at 160 hours/month, paid 2000 euros/month. To meet demand, company must recruit and retain new consultants. Training takes one month and 50 hours of supervision from an existing consultant. Trainees are paid 1000 euros/month. After training, 5% of the trainees leave the firm.

Plan the activity to meet the demand and incur the minimum cost.

## Variables

Training is 1 month, so trainees from the first month become consultants in the second month. 5% of trainees leave the firm, so 95% become consultants next month. In conclusion, the number of consultants from each month can be determined if the number of trainees is known, so the only variables of the problem will be number of trainees from each month. Since the estimation is done for only 5 months, it is not profitable to have trainees in the last month.

- $t_1$ - number of trainees in the first month
- $t_2$ - number of trainees in the second month
- $t_3$ - number of trainees in the third month
- $t_4$ - number of trainees in the fourth month

## Objective Function

The cost per month is $2000 \cdot number\_of\_consultants + 1000 \cdot number\_of\_trainees$.

Cost for each month:
- first month: $50 \cdot 2000 + t_1 \cdot 1000$
- second month: $(50 + 0.95t_1) \cdot 2000 + t_2 \cdot 1000$
- third month: $(50 + 0.95 \cdot (t_1 + t_2)) \cdot 2000 + t_3 \cdot 1000$
- fourth month: $(50 + 0.95 \cdot (t_1 + t_2 + t_3)) \cdot 2000 + t_4 \cdot 1000$
- fifth month: $(50 + 0.95 \cdot (t_1 + t_2 + t_3 + t_4)) \cdot 2000$

After doing the math, the minimization function is:

$min((5 \cdot 50 + 0.95 \cdot (4t_1 + 3t_2 + 2t_3 + t_4)) \cdot 2000 + (t_1 + t_2 + t_3 + t_4) \cdot 1000)$

## Constraints

- $50 \cdot 160 - t_1 \cdot 50 \geq 6000$, First Month Constraint
- $(50 + 0.95t_1) \cdot 160 - t_2 \cdot 50 \geq 7000$, First Month Constraint
- $(50 + 0.95 \cdot(t_1 + t_2)) \cdot 160 - t_3 \cdot 50 \geq 8000$, First Month Constraint
- $(50 + 0.95 \cdot(t_1 + t_2 + t_3)) \cdot 160 - t_4 \cdot 50 \geq 9500$, First Month Constraint
- $(50 + 0.95 \cdot(t_1 + t_2 + t_3 + t_4)) \cdot 160 - t_5 \cdot 50 \geq 11000$, First Month Constraint

## Code

In [2]:
from pulp import *

working_hours = 160
training_hours = 50
hours_target = [6000, 7000, 8000, 9500, 11000]

problem = LpProblem('Problem4', LpMinimize)

# Variables
t_vars = LpVariable.dicts('t', range(1, 5), lowBound=0, cat=LpInteger)

# Objective Function
problem += (5 * 50 + 0.95 * (4 * t_vars[1] + 3 * t_vars[2] + 2 * t_vars[3] + t_vars[4])) * 2000 + lpSum(t_vars) * 1000

# Constraints
problem += 50 * 160 - t_vars[1] * 50 >= hours_target[0], 'First Month Constraint'
problem += (50 + 0.95 * t_vars[1]) * 160 - t_vars[2] * 50 >= hours_target[1], 'Second Month Constraint'
problem += (50 + 0.95 * (t_vars[1] + t_vars[2])) * 160 - t_vars[3] * 50 >= hours_target[2], 'Third Month Constraint'
problem += (50 + 0.95 * (t_vars[1] + t_vars[2] + t_vars[3])) * 160 - t_vars[4] * 50 >= hours_target[3], 'Fourth Month Constraint'
problem += (50 + 0.95 * (t_vars[1] + t_vars[2] + t_vars[3] + t_vars[4])) * 160 >= hours_target[4], 'Fifth Month Constraint'

# Solve Problem and Display Results
problem.solve(PULP_CBC_CMD(msg=0))

if LpStatus[problem.status] == "Optimal":
    print(f'Total cost: ${value(problem.objective)}\n')

for variable in problem.variables():
    print(f'{variable.name}: {variable.varValue}')


Total cost: $590300.0

t_1: 0.0
t_2: 4.0
t_3: 9.0
t_4: 7.0
